# Import pacakage

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install Sastrawi
import re
import nltk 
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
nltk.download()

In [ ]:
from collections import Counter
from nltk import ngrams
from nltk import ngrams
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns

# Read data

In [ ]:
ig=pd.read_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_withoutstemming_label.csv')

In [ ]:
ig['label_score']=ig['label_score'].apply(lambda x:eval(x))
ig['label']=ig['label_score'].apply(lambda x:x['label'])
ig['score']=ig['label_score'].apply(lambda x:x['score'])

In [ ]:
ig = pd.read_excel('/content/drive/MyDrive/satria_data_penyisihan_2/BPJS_Instagram_2022-2021.xlsx')


In [ ]:
ig[['Konteks','Waktu_Post','Komentar']].head()

In [ ]:
ig['tahun']=ig['Waktu_Post'].dt.year
ig['bulan']=ig['Waktu_Post'].dt.month

In [ ]:
ig_post=ig.drop_duplicates(subset=['Konteks'])
ig_post=ig_post[ig_post.tahun.isin([2021,2022])]

In [ ]:
ig_post.shape

In [ ]:
ig_post['tahun'].value_counts()

In [ ]:
sns.countplot(ig_post.tahun)
plt.title('Total Post oleh BPJS Tiap Tahun')
plt.show()

In [ ]:
ig_post

In [ ]:
ig

# Sentiment analysis

In [ ]:
ig['label'].value_counts()

In [ ]:
ig[ig.tahun==2021]['label'].value_counts()

In [ ]:
ig[ig.tahun==2022]['label'].value_counts()

# Stopword removal

In [ ]:
listStopword=list(stopwords.words('indonesian'))
listStopword.extend(['cek','dm','kak','min','BPJS','gk','bpjskesehanri','dm','mgkn','ka','respon','sehat','bpjskesehatanri','bpjs','pandawa','indonesia','bls','kk','wa','sya','suami'])

In [ ]:
def remove_stopwords(sentence):
  words = sentence.split()
  words = [word for word in words if word not in listStopword and word.isalpha()]
  return " ".join(words)

In [ ]:
from PIL import Image


def wordcloud(sentence, title):
  sentence = " ".join(sentence)
  WC = WordCloud(background_color = 'white', max_words= 40)
  cloud = WC.generate(sentence)
  plt.figure(figsize=(15,10))
  plt.imshow(cloud, interpolation = "bilinear")
  plt.title(title)
  plt.axis("off")
  plt.show()

In [ ]:
ig.to_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_final.csv', index = False) # file udah stemming ada label dan score

In [ ]:
ig=pd.read_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_final.csv')

In [ ]:
ig['Komentar']=ig['Komentar'].astype('str')

In [ ]:
neutral = ig[ig['label'] == 'Neutral'].reset_index(drop = True)
positive = ig[ig['label'] == 'Positive'].reset_index(drop = True)
negative = ig[ig['label'] == 'Negative'].reset_index(drop = True)

# World cloud

## positive world cloud

In [ ]:
wordcloud(positive['Komentar'].apply(remove_stopwords), "Word Cloud Untuk Sentimen Positif")  


## negative world cloud

In [ ]:
wordcloud(negative['Komentar'].apply(remove_stopwords), "Word Cloud Untuk Sentimen negatif")


## neutral world cloud

In [ ]:
wordcloud(neutral['Komentar'].apply(remove_stopwords) ,"Word Cloud Untuk Sentimen neutral")

# Deteksi kata adjektif

In [ ]:
!pip install --upgrade flair

In [ ]:
import flair.datasets
corpus = flair.datasets.UD_INDONESIAN()

In [ ]:
tag_type = 'upos'
upos_dictionary = corpus.make_label_dictionary(label_type=tag_type)

In [ ]:
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, BertEmbeddings
from typing import List
embedding_types = [WordEmbeddings('id-crawl'),WordEmbeddings('id')]
embeddings = StackedEmbeddings(embeddings=embedding_types)

In [ ]:
from flair.models import SequenceTagger
tagger = SequenceTagger(hidden_size=256,embeddings=embeddings,
                                      tag_dictionary=upos_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

In [ ]:
from flair.trainers import ModelTrainer
trainer = ModelTrainer(tagger, corpus)
trainer.train('resources/taggers/example-universal-pos',
 learning_rate=0.1,
 mini_batch_size=32,
 max_epochs=10)

In [ ]:
!zip -r /content/drive/MyDrive/satria_data_penyisihan_2/pretrained.zip /content/resources

In [ ]:
from flair.data import Sentence
sentence = Sentence('saya dan dia kemarin pegi ke pasar bersama untuk membeli jeru')
tag_pos = SequenceTagger.load('resources/taggers/example-universal-pos/best-model.pt')
tag_pos.predict(sentence)
print(sentence.to_tagged_string())

In [ ]:
tag_pos = SequenceTagger.load('resources/taggers/example-universal-pos/best-model.pt')

In [ ]:
def find_adjective(sentence):
  sentence_array = Sentence(sentence)
  tag_pos.predict(sentence_array)
  adj_words = re.findall('"\w+"/ADJ', sentence_array.to_tagged_string())
  adj_words = [re.sub('/ADJ','', word) for word in adj_words]
  adj_words = [re.sub('"','', word) for word in adj_words]
  return " ".join(adj_words)

In [ ]:
ig.info()

In [ ]:
ig['adjektif'] =ig['Komentar'].apply(lambda text: find_adjective(text))

In [ ]:
ig.to_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_final_adjektif.csv', index = False)

In [ ]:
ig= pd.read_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_final_adjektif.csv')

In [ ]:
ig.dropna(subset=['Komentar'],inplace=True)

In [ ]:
ig.info()

In [ ]:
ig.dropna(inplace=True)

In [ ]:
ig=ig.reset_index(drop=True)

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

pretrained_name = "ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa"
nlp = pipeline("sentiment-analysis", model=pretrained_name, tokenizer=pretrained_name)

In [ ]:
ig['label_score_adjektif']=ig['adjektif'].apply(lambda x: nlp(x)[0])

In [ ]:
ig.to_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_final_adjektif_label.csv', index = False)

In [ ]:
tweet['Adjective Words']

In [ ]:
neutral = tweet[tweet['Label'] == 'Neutral'].reset_index(drop = True)
positive = tweet[tweet['Label'] == 'Positive'].reset_index(drop = True)
negative = tweet[tweet['Label'] == 'Negative'].reset_index(drop = True)

In [ ]:
top_20_neutral = word_frequency(neutral['Tweet Cleaned Lemmatized'], 2)[:40]
top_20_positive = word_frequency(positive['Tweet Cleaned Lemmatized'], 2)[:40]
top_20_negative = word_frequency(negative['Tweet Cleaned Lemmatized'], 2)[:40]

#### Positive

In [ ]:
kalimat = 'Aku suka makan'

In [ ]:
top_20_positive

In [ ]:
wordcloud(positive['Adjective Words'], "Word Cloud Untuk Sentimen Positif")

#### Negative

In [ ]:
wordcloud(negative['Adjective Words'], "Word Cloud Untuk Sentimen Negative")

# word frequency

In [ ]:
def word_frequency(sentence, grams):
  sentence = " ".join(sentence)
  new_tokens = nltk.word_tokenize(sentence)
  counted = Counter(ngrams(new_tokens, grams))
  word_freq = pd.DataFrame(counted.items(), columns = ['Kata','Frekuensi']).sort_values(by = 'Frekuensi',ascending = False)
  return word_freq

## positive

In [ ]:
word_frequency(positive['Komentar'].apply(remove_stopwords),2).head(40).reset_index(drop=True)

## negative

In [ ]:
word_frequency(negative['Komentar'].apply(remove_stopwords),2).head(40).reset_index(drop=True)

# Analisa tersendiri

In [ ]:
ig=pd.read_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_withoutstemming_label.csv')
ig['label_score']=ig['label_score'].apply(lambda x:eval(x))
ig['label']=ig['label_score'].apply(lambda x:x['label'])
ig['score']=ig['label_score'].apply(lambda x:x['score'])

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(x=ig.label)
plt.title('Hasil Predisik Model Bert')
plt.xlabel('Hasil Prediksi')
plt.show()

In [ ]:
ig.info()

In [ ]:
ig.info()

In [ ]:
ig[['Komentar','tahun','month']].info()

In [ ]:
ig.info()

# Analisa kata sifat

In [ ]:
ig= pd.read_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_final_adjektif.csv')

In [ ]:
ig.dropna(inplace=True)

In [ ]:
ig